# CSCI 5523 Introduction of Data Mining
## Assignment 1
### Jooyong Lee

In [1]:
# Import necessary packages.
import pyspark

In [2]:
# Create SparkContect object.
if __name__ == '__main__':
    
    sc_conf = pyspark.SparkConf() \
        .setAppName('task1') \
        .setMaster('local[*]') \
        .set('spark.driver.memory', '8g') \
        .set('spark.executer.memory', '4g')
    
    sc = pyspark.SparkContext(conf=sc_conf)
    sc.setLogLevel('OFF')

In [3]:
# Read review.json as a textfile.
review_rdd = sc.textFile('/Users/jooyong/Downloads/review.json')

### 4.1 Task1: Data Exploration
#### 4.1.1 
#### (A)

In [4]:
# Every line include one sample, so seperate txt with 'changes of a line'.
length_review_rdd = review_rdd.map(lambda x: (x.split('\b')[0], 1)).count()

In [5]:
# It will print the number of lines, which is the number of samples(reviews).
print('The number of reviews is', length_review_rdd)

The number of reviews is 1151625


#### (B)

In [6]:
# Mapping to make (year of the review, 1) key:value pairs.
year_review_rdd = review_rdd.map(lambda x: (eval(x.split('\b')[0])['date'][:4], 1))

In [7]:
# Let's observe the first 10 samples.
year_review_rdd.take(10)

[('2017', 1),
 ('2018', 1),
 ('2015', 1),
 ('2017', 1),
 ('2017', 1),
 ('2016', 1),
 ('2017', 1),
 ('2018', 1),
 ('2017', 1),
 ('2014', 1)]

In [8]:
# Reducing by key(year of the review) to make (year of the review, counts) key:value pairs.
grouped_year_review_rdd = year_review_rdd.reduceByKey(lambda a, b:a+b)

In [9]:
# Let's observe samples. We can observe the number of reviews(value) depends on the given year(key).
grouped_year_review_rdd.collect()

[('2005', 140),
 ('2014', 121373),
 ('2017', 209995),
 ('2009', 17288),
 ('2013', 84560),
 ('2011', 52224),
 ('2016', 189189),
 ('2012', 63540),
 ('2010', 32077),
 ('2006', 891),
 ('2007', 3627),
 ('2004', 3),
 ('2018', 202784),
 ('2015', 164146),
 ('2008', 9788)]

#### (C)

In [10]:
# Mapping to make (user_id, 1) key:value pairs.
user_review_rdd = review_rdd.map(lambda x: (eval(x.split('\b')[0])['user_id'], 1))

In [11]:
# It will print the number of distinct users.
print('The number of distinct users is:', user_review_rdd.distinct().count())

The number of distinct users is: 566269


#### (D)

In [12]:
# Reducing by key(user_id of the review) to make (user_id of the review, counts) key:value pairs.
grouped_user_review_rdd=user_review_rdd.reduceByKey(lambda a, b:a+b)

In [13]:
# Order tuples in descending order and print out first 15 of users who have the largest reviews.
grouped_user_review_rdd.sortBy(lambda x:x[1], ascending=False).take(15)

[('CxDOIDnH8gp9KXzpBHJYXw', 715),
 ('bLbSNkLggFnqwNNzzq-Ijw', 424),
 ('PKEzKWv_FktMm2mGPjwd0Q', 322),
 ('DK57YibC5ShBmqQl97CKog', 291),
 ('ELcQDlf69kb-ihJfxZyL0A', 288),
 ('U4INQZOPSUaj8hMjLlZ3KA', 276),
 ('QJI9OSEn6ujRCtrX06vs1w', 258),
 ('d_TBs6J3twMy9GChqUEXkg', 253),
 ('hWDybu_KvYLSdEFzGrniTw', 239),
 ('dIIKEfOgo0KqUfGQvGikPg', 216),
 ('UYcmGbelzRa0Q6JqzLoguw', 210),
 ('n86B7IkbU20AkxlFX_5aew', 208),
 ('YRcaNlwQ6XXPFDXWtuMGdA', 207),
 ('rCWrxuRC8_pfagpchtHp6A', 203),
 ('62GNFh5FySkA3MbrQmnqvg', 200)]

#### (E)

In [14]:
# Read given stopwords and make the list of it.
stopwords = open('/Users/jooyong/Downloads/stopwords')
list_stopwrods = stopwords.read().split("\n")

In [25]:
# Make a list of punctuations to replace it to empty space.
table = str.maketrans("(){}[],./?!~@#$%^&*()_+-=:;'<>|", "                               ")

# Mapping to make (word, 1) key:value pairs.
text_review_rdd = review_rdd.flatMap(lambda x: eval(x.split('\b')[0])['text'].translate(table).split(" ")). \
                  map(lambda word: (word.lower(),1))

In [26]:
# Let's observe first 10 samples.
text_review_rdd.take(10)

[('walked', 1),
 ('in', 1),
 ('around', 1),
 ('4', 1),
 ('on', 1),
 ('a', 1),
 ('friday', 1),
 ('afternoon', 1),
 ('', 1),
 ('we', 1)]

In [27]:
# Reducing by key(word) to make (word, counts) key:value pairs.
grouped_text_review_rdd = text_review_rdd.reduceByKey(lambda a, b: a+b)

In [28]:
# Let's observe first 10 samples.
grouped_text_review_rdd.take(10)

[('', 19035555),
 ('even', 221288),
 ('they', 1056404),
 ('noticed', 19062),
 ('not', 778025),
 ('for', 1524958),
 ('well', 200407),
 ('from', 364129),
 ('ever', 116449),
 ('sure', 115352)]

In [29]:
# Sort by key values in descending order.
sorted_grouped_text_review_rdd = grouped_text_review_rdd.sortBy(lambda x:x[1], ascending=False).collect()

In [30]:
# exclude tuples includes a stopword.
for i in sorted_grouped_text_review_rdd[:]:
    if i[0] in list_stopwrods:
        sorted_grouped_text_review_rdd.remove(i)

In [32]:
# Top 30 frequent words.
sorted_grouped_text_review_rdd[:30]

[('food', 570906),
 ('place', 561700),
 ('good', 558467),
 ('great', 493678),
 ('like', 407575),
 ('service', 404218),
 ('time', 401935),
 ('get', 384771),
 ('one', 374984),
 ('would', 356249),
 ('back', 346633),
 ('go', 312129),
 ('really', 291100),
 ('also', 243432),
 ('us', 240380),
 ('ve', 231404),
 ('got', 226852),
 ('even', 221288),
 ('nice', 211832),
 ('m', 203185),
 ('well', 200407),
 ('best', 197236),
 ('always', 185229),
 ('came', 184227),
 ('ordered', 181675),
 ('staff', 180179),
 ('love', 179673),
 ('didn', 178044),
 ('order', 176664),
 ('first', 172272)]